# Setup

## Imports

In [ ]:
# Install-Module -Name DattoRMM -Force -AllowClobber -Verbose
# Install-Module Microsoft.Graph.Calendar -Force -AllowClobber
# Install-Module Microsoft.Graph.Authentication -Force -AllowClobber
# Install-Module Microsoft.Graph.Mail -Force -AllowClobber
# Install-Module Microsoft.Graph.Teams -Force -AllowClobber
# Install-Module Microsoft.Graph.Users -Force -AllowClobber
# Install-Module PowerHTML -Force -AllowClobber
# Install-Module Pode -Force -AllowClobber
# Install-Module Pode.Web -Force -AllowClobber
# $HtmlAgilityPack = "C:\Users\David\OneDrive - Avartec Inc\Documents\Clients\Avartec\nuget\lib\Net45\HtmlAgilityPack.dll"
# Add-Type -Path $HtmlAgilityPack
# Install-Package Newtonsoft.Json -Force -AllowClobber


. ".\Powershell\Get-StoredObject.ps1"
. ".\Powershell\Get-StoredCredential.ps1"
Import-Module DattoRMM
Import-Module Microsoft.Graph.Authentication
Import-Module Microsoft.Graph.Calendar
Import-Module Microsoft.Graph.Mail
Import-Module Microsoft.Graph.Teams
Import-Module Microsoft.Graph.Users
Add-Type -AssemblyName System.Web
Import-Module PowerHTML
Add-Type -AssemblyName Newtonsoft.Json

Connect-MgGraph -Scopes "Mail.Read", "MailboxFolder.Read", "MailboxItem.Read", "Mail.ReadBasic", "MailboxSettings.Read", "Calendars.Read", "Chat.ReadBasic", "Chat.Read", "ChatMessage.Read", "TeamsAppInstallation.ReadForChat", "Channel.ReadBasic.All", "Team.ReadBasic.All", "TeamsActivity.Read", "Files.Read", "Files.Read.All" -NoWelcome


## Variables

In [ ]:
# ### Microsoft
# $userId = "david@avartec.com"
# $userDisplayName = "David Midlo"
# $tenantId = ""

# $graphCreds = Get-StoredCredential -CredentialName "MicrosoftGraph" -Path ".\StoredCredentials"
# Connect-MgGraph -Scopes "Mail.Read", "MailboxFolder.Read", "MailboxItem.Read", "Mail.ReadBasic", "MailboxSettings.Read", "Calendars.Read", "Chat.ReadBasic", "Chat.Read", "ChatMessage.Read", "TeamsAppInstallation.ReadForChat", "Channel.ReadBasic.All", "Team.ReadBasic.All", "TeamsActivity.Read", "Files.Read", "Files.Read.All" -NoWelcome

# $msGraphUser = Get-MgUser -UserId $userId


# $sentItemsFolder = Get-MgUserMailFolder -UserId $userId -MailFolderId "sentitems"


### Get Date Ranges for API Throttling - Each Day

In [34]:
function Get-SortedDateRanges {
    param (
        [Parameter(Mandatory = $true)]
        $startDateTime,
        
        [Parameter(Mandatory = $true)]
        $endDateTime
    )
    # Initialize an empty hashtable for date ranges
    $dateRanges = @{}

    # Initialize a counter to label each block
    $blockCounter = 1

    # Iterate through the date range in four-hour increments
    $currentBlockStart = $startDateTime
    while ($currentBlockStart -lt $endDateTime) {
        # Define the end time for the current block as four hours later
        $currentBlockEnd = $currentBlockStart.AddHours(24)

        # If the end of the block exceeds the end date, cap it at endDateTime
        if ($currentBlockEnd -gt $endDateTime) {
            $currentBlockEnd = $endDateTime
        }

        # Pad the blockCounter with leading zeros to 5 digits
        $blockLabel = "block" + $blockCounter.ToString().PadLeft(5, '0')

        # Add the start and end time to the dateRanges hashtable
        $dateRanges[$blockLabel] = @{
            StartTime = $currentBlockStart
            EndTime   = $currentBlockEnd
        }

        # Move to the next block
        $currentBlockStart = $currentBlockEnd
        $blockCounter++
    }

    # Display the result
    # Convert the hashtable to an ordered dictionary to maintain the sort order
    $sortedDateRanges = [ordered]@{}

    # Sort by the block name and populate the ordered dictionary
    foreach ($key in ($dateRanges.Keys | Sort-Object)) {
        $sortedDateRanges[$key] = $dateRanges[$key]
    }

    return $sortedDateRanges
}


# Services Data Cleaning

## ActivityWatch

### Import Activity Watch CSVs into Object

In [35]:
function Get-DirectoryCSVs {
    param (
        [Parameter(Mandatory = $true)]
        $directoryPath
    )

    return (Get-ChildItem -Path $directoryPath -Filter *.csv)
}

function Import-CSVsIntoObject {
    param (
        [Parameter(Mandatory = $true)]
        $fileBlob
    )

    $csvObjects = @()
    foreach ($file in $fileBlob) {
        $csvContent = Import-Csv -Path $file.FullName

        $csvObject = [PSCustomObject]@{
            FileName = $file.Name
            FilePath = $file.FullName
            FileContent = $csvContent
        }
        $csvObjects += $csvObject
    }
    return $csvObjects
}

function Import-ActivityWatchCSVs {
    param (
        [Parameter(Mandatory = $true)]
        $activitywatchDirectory
    )

    $ActivityWatchCSVs = Get-DirectoryCSVs -directoryPath $activitywatchDirectory
    
    return (Import-CSVsIntoObject -fileBlob $ActivityWatchCSVs)
}


### Fixing Obsidian Watcher CSV

In [36]:
function Consolodate-ActivityWatchObsidianEvents {
    param (
        [Parameter(Mandatory = $true)]
        $obsidianCsv
    )

    $processedRows = @()
    $sortedCSV = $obsidianCsv | Sort-Object {[datetime]$_."timestamp"}
    $currentBlock = $null  # To keep track of the current block

    foreach ($row in $sortedCSV) {
        # Parse the timestamp
        $currentTimestamp = [datetime]::Parse($row.timestamp)

        if ($currentBlock -eq $null) {
            # Start a new block
            $currentBlock = @{
                StartRow = $row
                EndRow = $row
                Duration = 0
                StartTimestamp = $currentTimestamp
                EndTimestamp = $currentTimestamp
            }
        } else {
            # Check if the current row matches the current block
            $isSameSession =    ($row.file -eq $currentBlock.EndRow.file) -and
                                ($row.project -eq $currentBlock.EndRow.project) -and
                                ($row.language -eq $currentBlock.EndRow.language) -and
                                ($row.projectPath -eq $currentBlock.EndRow.projectPath) -and
                                ($row.editor -eq $currentBlock.EndRow.editor) -and
                                ($row.editorVersion -eq $currentBlock.EndRow.editorVersion) -and
                                ($row.eventType -eq $currentBlock.EndRow.eventType)

            if ($isSameSession) {
                # Update the current block
                $timeDifference = $currentTimestamp - $currentBlock.EndTimestamp
                $currentBlock.Duration += $timeDifference.TotalSeconds
                $currentBlock.EndRow = $row
                $currentBlock.EndTimestamp = $currentTimestamp
            } else {
                # Finalize the current block and add to processed rows
                $summarizedRow = $currentBlock.StartRow.PSObject.Copy()
                $summarizedRow.timestamp = $currentBlock.StartTimestamp.ToString("o")
                $summarizedRow.Duration = $currentBlock.Duration
                $processedRows += $summarizedRow

                # Start a new block
                $currentBlock = @{
                    StartRow = $row
                    EndRow = $row
                    Duration = 0
                    StartTimestamp = $currentTimestamp
                    EndTimestamp = $currentTimestamp
                }
            }
        }
    }

    # After processing all rows, don't forget to add the last block
    if ($currentBlock -ne $null) {
        $summarizedRow = $currentBlock.StartRow.PSObject.Copy()
        $summarizedRow.timestamp = $currentBlock.StartTimestamp.ToString("o")
        $summarizedRow.Duration = $currentBlock.Duration
        $processedRows += $summarizedRow
    }

    return ($processedRows | Where-Object {$_.duration -gt 0})
}

function Repair-ChildActivityWatchObsidianCSV {
    param (
        [Parameter(Mandatory = $true)]
        $csvObjects
    )

    foreach ($csvObject in $csvObjects) {
        if ($csvObject.FileName -like "*obsidian*") {
            $csvObject.FileContent = Consolodate-ActivityWatchObsidianEvents -obsidianCsv $csvObject.FileContent
        }
    }

    return $csvObjects
}


### Applying ActivityWatch AFK data

In [37]:
function Import-ActivityWatchAFKs {
    param (
        [Parameter(Mandatory = $true)]
        $activitywatchAfkDirectory
    )

    $AfkCSVs = Get-DirectoryCSVs -directoryPath $activitywatchAfkDirectory

    return {Import-CSVsIntoObject -fileBlob $AfkCSVs}
}


In [38]:
function Filter-ActivityWatchByAfkStatus {
    param (
        [Parameter(Mandatory = $true)]
        $activityWatchCSVsObject,

        [Parameter(Mandatory = $true)]
        $storeObjectPath,

        [Parameter(Mandatory = $true)]
        $afkCSVsDirectory
    )

    $object = Get-StoredObject -ObjectName "afkFilteredActivityWatch" -Path $storeObjectPath

    if ($object) {
        $csvObjects = $object
    } else {
        $csvObjects = $activityWatchCSVsObject
        $afkObjects = Import-ActivityWatchAFKs -activitywatchAfkDirectory $afkCSVsDirectory

        foreach ($csvObject in $csvObjects){

            $filteredContent = @()
            foreach ($csvEvent in $csvObject.FileContent) {
                $count++
                $csvStartTime = [DateTime]::Parse($csvEvent.timestamp)
                $csvDuration = New-TimeSpan -Seconds ([int]([Math]::Ceiling($csvEvent.duration)))
                $csvEndTime = $csvStartTime + $csvDuration
                $eventOverlap = $false

                foreach ($file in $afkObjects) {
                    $afkCsv = $file.FileContent
                    $Afk = $afkCsv | Where-Object {$_.status -like "afk"}


                    foreach ($afkEvent in $Afk){
                        $afkStartTime = [DateTime]::Parse($afkEvent.timestamp)
                        $afkDuration = New-TimeSpan -Seconds ([int]([Math]::Ceiling($afkEvent.duration)))
                        $afkEndTime = $afkStartTime + $afkDuration

                        if ($csvStartTime -le $afkStartTime -and $csvEndTime -ge $afkEndTime) {
                            # csv: |-------------|
                            # afk:   |---------|
                            $eventOverlap = $true
                            # Write-Host "Full overlap: CSV interval fully contains AFK interval. Creating two entries"
                            # Write-Host "cs: $csvStartTime as: $afkStartTime ae: $afkEndTime ce: $csvEndTime"
                            # Write-Host "-------------------------------------"

                            $frontEvent = $csvEvent.PSObject.Copy()
                            $frontEventStartTime = $csvStartTime
                            $frontEventEndTime = $afkStartTime.AddSeconds(-1)
                            $frontEvent.duration = ($frontEventEndTime - $frontEventStartTime).TotalSeconds
                            $frontEvent.timestamp = $frontEventStartTime.ToString("yyyy-MM-ddTHH:mm:ss.fffK")
                            $filteredContent += $frontEvent

                            $backEvent = $csvEvent.PSObject.Copy()
                            $backEventStartTime = $afkEndTime.AddSeconds(1)
                            $backEventEndTime = $csvEndTime
                            $backEvent.duration = ($backEventEndTime - $backEventStartTime).TotalSeconds
                            $backEvent.timestamp = $backEventStartTime.ToString("yyyy-MM-ddTHH:mm:ss.fffK")
                            $filteredContent += $backEvent

                            break
                        }
                        elseif ($afkStartTime -le $csvStartTime -and $afkEndTime -ge $csvEndTime) {
                            # csv:   |---------|
                            # afk: |-------------|
                            $eventOverlap = $true
                            # Write-Host "Full overlap: AFK interval fully contains CSV interval. Removing Entry"
                            # Write-Host "as: $afkStartTime cs: $csvStartTime ce: $csvEndTime ae: $afkEndTime"
                            # Write-Host "-------------------------------------"
                            break
                        }
                        elseif ($csvStartTime -le $afkStartTime -and $csvEndTime -ge $afkStartTime -and $csvEndTime -le $afkEndTime) {
                            # csv: |---------|
                            # afk:     |---------|
                            $eventOverlap = $true
                            # Write-Host "Partial overlap at start. Trimming Entry"
                            # Write-Host "cs: $csvStartTime as: $afkStartTime ce: $csvEndTime ae: $afkEndTime"
                            # Write-Host "-------------------------------------"

                            $frontEvent = $csvEvent.PSObject.Copy()
                            $frontEventStartTime = $csvStartTime
                            $frontEventEndTime = $afkStartTime.AddSeconds(-1)
                            $frontEvent.duration = ($frontEventEndTime - $frontEventStartTime).TotalSeconds
                            $frontEvent.timestamp = $frontEventStartTime.ToString("yyyy-MM-ddTHH:mm:ss.fffK")
                            $filteredContent += $frontEvent
                            break
                        }
                        elseif ($csvStartTime -ge $afkStartTime -and $csvStartTime -le $afkEndTime -and $csvEndTime -ge $afkEndTime) {
                            # csv:     |---------|
                            # afk: |---------|
                            $eventOverlap = $true
                            # Write-Host "Partial overlap at end. Trimming Entry"
                            # Write-Host "as: $afkStartTime cs:$csvStartTime ae: $afkEndTime ce: $csvEndTime"
                            # Write-Host "-------------------------------------"

                            $backEvent = $csvEvent.PSObject.Copy()
                            $backEventStartTime = $afkEndTime.AddSeconds(1)
                            $backEventEndTime = $csvEndTime
                            $backEvent.duration = ($backEventEndTime - $backEventStartTime).TotalSeconds
                            $backEvent.timestamp = $backEventStartTime.ToString("yyyy-MM-ddTHH:mm:ss.fffK")
                            $filteredContent += $backEvent
                            break
                        }
                        else {}

                    }
                }

                if (-not $eventOverlap) {
                    $filteredContent += $csvEvent
                }
            }
            $csvObject.FileContent = $filteredContent
        }

        Get-StoredObject -ObjectName "afkFilteredActivityWatch" -Path $storeObjectPath -InputObject $csvObjects
    }

    return $csvObjects
}


## Datto

### Get Datto Activity Logs

In [39]:
function Get-DattoRemoteActivityLog {
    param (
        [Parameter(Mandatory = $true)]
        [DateTime]$startDateTime,

        [Parameter(Mandatory = $true)]
        [DateTime]$endDateTime,

        [Parameter(Mandatory = $true)]
        $dattoUserId,

        [Parameter(Mandatory = $true)]
        $storedObjectPath
    )

    $objectName = "$dattoUserId-$($startDateTime.ToString("yyyyMMdd'T'HHmmssK"))-to-$($endDateTime.ToString("yyyyMMdd'T'HHmmssK"))-datto"

    $storedLogs = Get-StoredObject -ObjectName $objectName -Path $storedObjectPath

    if ($storedLogs) {
        $dattoActivityLogs = $storedLogs
    } else {

        $dattoActivityLogs = $null
        $dattoActivityLogs = @()

        $logRequest = Get-DrmmActivityLogs -StartDateTime $startDateTime -EndDateTime $endDateTime -Categories "remote" -UserIds $dattoUserId 
        foreach ($entry in $logRequest) {
            $entryCount++
            $entryStartTime = $entry.details.'remote_session.start_date'
            $entryEndTime = $entry.details.'remote_session.end_date'
            $duration = ($entryEndTime - $entryStartTime).TotalSeconds
            
            $log = @{
                timestamp = $entryStartTime.ToString("yyyy-MM-ddTHH:mm:ss.fffZ")
                duration = $duration
                category = $entry.category
                siteName = $entry.siteName
                hostname = $entry.details.'device.hostname'
                action = $entry.details.'event.action'
                sessionType = $entry.details.'remote_session.type'
            }

            $dattoActivityLogs += $log

            Start-Sleep -Seconds 1
        }

        if ($dattoActivityLogs -ne $null) {
            Get-StoredObject -ObjectName $objectName -Path $storedObjectPath -InputObject $dattoActivityLogs | Out-Null
        }
    }
    return $dattoActivityLogs
}

function Get-DattoRemoteActivityCsvObject {
    param (
        [Parameter(Mandatory = $true)]
        [DateTime]$startDateTime,

        [Parameter(Mandatory = $true)]
        [DateTime]$endDateTime,

        [Parameter(Mandatory = $true)]
        $dattoUserId,

        [Parameter(Mandatory = $true)]
        $storedObjectPath
    )

    $objectName = "$dattoUserId-$($startDateTime.ToString("yyyyMMdd'T'HHmmssK"))-to-$($endDateTime.ToString("yyyyMMdd'T'HHmmssK"))-datto"
    $dattoActivityLog = Get-DattoRemoteActivityLog -startDateTime $startDateTime -endDateTime $endDateTime -dattoUserId $dattoUserId -storedObjectPath $storedObjectPath

    if ($dattoActivityLog -ne $null) {
        $csvObject = [PSCustomObject]@{
            FileName = $objectName
            FilePath = $objectName
            FileContent = ($dattoActivityLog | ConvertTo-Csv) | ConvertFrom-Csv -ErrorAction SilentlyContinue
        }

        return $csvObject
    } else {
        return $false
    }
}

function Add-DattoRemoteActivityCSVs {
    param (
        [Parameter(Mandatory = $true)]
        $dattoUserId,

        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $csvObjects,

        [Parameter(Mandatory = $true)]
        $dateRangesObject
    )

    $dattoUrl = "https://concord-api.centrastage.net"
    $dattoCreds = Get-StoredCredential -CredentialName "DattoRMM" -Path ".\StoredCredentials"
    Set-DrmmApiParameters -Url $dattoUrl -Credential $dattoCreds

    foreach ($range in $dateRangesObject.GetEnumerator()) {
        
        $csvObject = Get-DattoRemoteActivityCsvObject -startDateTime $range.Value.StartTime -endDateTime $range.Value.EndTime -dattoUserId $dattoUserId -storedObjectPath $storedObjectPath
        
        if ($csvObject) {
            $csvObjects += $csvObject
        }

    }

    return $csvObjects
}


## Office Online Calendar

### Get Calendar Events from Office Online

In [40]:
function Get-OfficeOnlineCalendarEvents {
    param (
        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $dateRangesObject,

        [Parameter(Mandatory = $true)]
        $userId
    )
    $jobs = @()

    $object = Get-StoredObject -ObjectName "MicrosoftCalendarEventIds" -Path $storedObjectPath

    if ($object) {
        $allResults = $object
    } else {
        $defaultCalendar = Get-MgUserCalendar -UserId $userId | Where-Object { $_.Name -eq "Calendar" } | Select-Object -First 1

        foreach ($range in $dateRangesObject.GetEnumerator()) {
            $jobs += Start-ThreadJob -ScriptBlock {
                param($startTime, $endTime, $userId, $defaultCalendar)

                function Get-CalendarEvents {
                    param($startTime, $endTime, $UserId)
                    try {
                        $Events = Get-MgUserCalendarEvent -UserId $UserId -Filter "start/dateTime ge '$startTime' and end/dateTime le '$endTime'" -CalendarId $defaultCalendar.Id
                        return $Events
                    } catch {
                        Write-Output "Error retrieving events for $startTime to $($endTime): $($_)"
                        return $null
                    }
                }

                Get-CalendarEvents -startTime $startTime -endTime $endTime -UserId $UserId
            } -ArgumentList $range.Value.StartTime, $range.Value.EndTime, $userId, $defaultCalendar
        }

        $allResults = @()
        foreach ($job in $jobs) {
            $result = Receive-Job -Job $job -Wait -ErrorAction SilentlyContinue
            if ($result) {
                $allResults += $result
            }
            Remove-Job -Job $job
        }

        Get-StoredObject -ObjectName "MicrosoftCalendarEventIds" -Path $storedObjectPath -InputObject $allResults
    }

    return $allResults
}


### Convert Office 365 Calendar Events

In [41]:
function Convert-OfficeOnlineCalendarEvents {
    param (
        [Parameter(Mandatory = $true)]
        $startDateTime,

        [Parameter(Mandatory = $true)]
        $endDateTime,

        [Parameter(Mandatory = $true)]
        $csvObjects,

        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $dateRangesObject,

        [Parameter(Mandatory = $true)]
        $userId
    )

    $objectName = "$userId-$($startDateTime.ToString("yyyyMMdd'T'HHmmssK"))-to-$($endDateTime.ToString("yyyyMMdd'T'HHmmssK"))-MicrosoftOnlineCalendar"

    $allResults = Get-OfficeOnlineCalendarEvents -storedObjectPath $storedObjectPath -dateRangesObject $dateRangesObject -userId $userId

    $calendarEventLogs = @()
    foreach ($row in $allResults) {
        [datetime]$calendarStartDateTime = [string]$row.Start.Datetime
        [datetime]$calendarEndDateTime = [string]$row.End.Datetime

        $body = [System.Web.HttpUtility]::HtmlDecode(($row.Body | ConvertFrom-Html).InnerText)
        $importance = $row.Importance
        $isAllDay = $row.IsAllDay
        $isDraft = $row.IsDraft
        $isOnlineMeeting = $row.IsOnlineMeeting
        $isOrganizer = $row.IsOrganizer
        $onlineMeetingProvider = $row.OnlineMeetingProvider
        $onlineMeetingUrl = $row.OnlineMeetingUrl
        $organizer = "$($row.Organizer.EmailAddress.Name) - $($row.Organizer.EmailAddress.Address)"
        $subject = $row.Subject
        $weblink = $row.WebLink

        $onlineMeeting = ""
        $onlineMeetingHeaders = $row.OnlineMeeting | Get-Member -MemberType Property | Where-Object {$_.Name -notin @("AdditionalProperties")} | Select-Object -ExpandProperty Name 
        foreach ($onlineMeetingHeader in $onlineMeetingHeaders){
            $onlineMeeting += "$onlineMeetingHeader $($row.OnlineMeeting.$onlineMeetingHeader), "
        }
        $onlineMeeting = $onlineMeeting.TrimEnd(", ")

        $location = ""
        $locationHeaders = $row.Location | Get-Member -MemberType Property |  Where-Object {$_.Name -notin @("AdditionalProperties", "UniqueId", "UniqueIdType")} | Select-Object -ExpandProperty Name 
        foreach ($locationHeader in $locationHeaders) {
            if ($row.Location.$locationHeader -ne $null) {

                switch(($row.Location.$locationHeader.GetType()).Name) {
                    "PSObject" {
                        $objectHeaders = $row.Location.$locationHeader | Get-Member -MemberType Property | Where-Object {$_.Name -notin @("AdditionalProperties", "Accuracy", "Altitude", "AltitudeAccuracy")} | Select-Object -ExpandProperty Name
                        foreach ($objectHeader in $objectHeaders){
                            $location += "$objectHeader - $($row.Location.$locationHeader.$objectHeader), "
                        }
                    }
                    "String" {
                        $location += "$($row.Location.$locationHeader), "
                    }
                }
            } 
        }
        $location = $location.TrimEnd(", ")

        $attendees = ""
        foreach ($row in $row.Attendees) {
            $attendeeName = $row.EmailAddress.Name
            $attendeeEmail = $row.EmailAddress.Address

            $attendees += "$attendeeName - $attendeeEmail, "
        }
        $attendees = $attendees.TrimEnd(", ")

        
        $log = [PSCustomObject]@{
            timestamp = $calendarStartDateTime.ToString("yyyy-MM-ddTHH:mm:ss.fffZ")
            duration = ($calendarEndDateTime - $calendarStartDateTime).TotalSeconds
            attendees = $attendees
            body = $body
            importance = $importance
            location = $location
            isAllDay = $isAllDay
            isDraft = $isDraft
            isOnlineMeeting = $isOnlineMeeting
            isOrganizer = $isOrganizer
            onlineMeeting = $onlineMeeting
            onlineMeetingProvider = $onlineMeetingProvider
            onlineMeetingUrl = $onlineMeetingUrl
            organizer = $organizer
            subject = $subject
            weblink = $weblink
        }

        $calendarEventLogs += $log
    }

    $csvObject = [PSCustomObject]@{
            FileName = $objectName
            FilePath = $objectName
            FileContent = ($calendarEventLogs | ConvertTo-Csv) | ConvertFrom-Csv -ErrorAction SilentlyContinue
        }
    $csvObjects += $csvObject

    return $csvObjects
}


## Office Online Sent Emails

### Get Sent Emails from Office Online

In [42]:
function Get-OfficeOnlineSentEmails {
    param (
        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $dateRangesObject,

        [Parameter(Mandatory = $true)]
        $userId
    )

    $jobs = @()

    $object = Get-StoredObject -ObjectName "MicrosoftSentEmailIds" -Path $storedObjectPath

    if ($object) {
        $allResults = $object
    } else {
        $sentItemsFolder = Get-MgUserMailFolder -UserId $userId -MailFolderId "sentitems"

        foreach ($range in $dateRangesObject.GetEnumerator()) {
            $jobs += Start-ThreadJob -ScriptBlock {
                param($startTime, $endTime, $userId, $sentItemsFolder)

                try {
                    # Retrieve all messages in the Sent Items folder
                    $messages = @()
                    $page = Get-MgUserMailFolderMessage -UserId $userId -MailFolderId $sentItemsFolder.Id -PageSize 100

                    $messages += $page

                    while ($page.NextPageLink) {
                        $page = Invoke-MgGraphRequest -Uri $page.NextPageLink -Method GET
                        $messages += $page.Value
                    }

                    # Filter messages by sentDateTime client-side
                    $filteredMessages = $messages | Where-Object {
                        ($_).sentDateTime -ge $startTime -and ($_.sentDateTime) -le $endTime
                    }

                    # Return the filtered messages
                    return $filteredMessages
                } catch {
                    Write-Output "Error retrieving sent emails from $startTime to $($endTime): $($_)"
                    return $null
                }
            } -ArgumentList $range.Value.StartTime, $range.Value.EndTime, $userId, $sentItemsFolder
        }

        $allResults = @()
        foreach ($job in $jobs) {
            $result = Receive-Job -Job $job -Wait -ErrorAction SilentlyContinue
            if ($result) {
                $allResults += $result
            }
            Remove-Job -Job $job
        }

        # Store the results
        Get-StoredObject -ObjectName "MicrosoftSentEmailIds" -Path $storedObjectPath -InputObject $allResults
    }

    return $allResults
}


### Convert Office Online Sent Emails

In [43]:
function Convert-OfficeOnlineSentEmails {
    param (
        [Parameter(Mandatory = $true)]
        $startDateTime,

        [Parameter(Mandatory = $true)]
        $endDateTime,

        [Parameter(Mandatory = $true)]
        $csvObjects,

        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $dateRangesObject,

        [Parameter(Mandatory = $true)]
        $userId
    )

    $objectName = "$userId-$($startDateTime.ToString("yyyyMMdd'T'HHmmssK"))-to-$($endDateTime.ToString("yyyyMMdd'T'HHmmssK"))-MicrosoftOnlineEmails"
    $allResults = Get-OfficeOnlineSentEmails -storedObjectPath $storedObjectPath -dateRangesObject $sortedDateRanges -userId $userId

    [PSObject]$sentEmails = @()
    foreach ($row in $allResults) {
        $weblink = $row.WebLink
        $subject = $row.Subject
        $isDraft = $row.IsDraft
        $inferenceClassification = $row.InferenceClassification
        $importance = $row.Importance
        $hasAttachments = $row.HasAttachments
        $flag = $row.Flag

        if ($row.Body.Content){
            $bodyText = [System.Web.HttpUtility]::HtmlDecode(($row.Body.Content | ConvertFrom-Html).InnerText)
        } else {
            $bodyText = ""
        }

        $ccRecipients = ""
        foreach ($ccRecipient in $row.CcRecipients){
            $ccRecipients += "$($ccRecipient.EmailAddress.Name) - $($ccRecipient.EmailAddress.Address), "
        }
        $ccRecipients.TrimEnd(", ") | Out-Null

        $bccRecipients = ""
        foreach ($bccRecipient in $row.BccRecipients) {
            $bccRecipients += "$($bccRecipient.EmailAddress.Name) - $($bccRecipient.EmailAddress.Address), "
        }
        $bccRecipients.TrimEnd(", ") | Out-Null

        $log = [PSCustomObject]@{
            timestamp = $row.SentDateTime.ToString("yyyy-MM-ddTHH:mm:ss.fffZ")
            duration = ($row.LastModifiedDateTime - $row.SentDateTime).TotalSeconds
            bccRecipients = $bccRecipients
            body = $bodyText
            flag = $flag
            hasAttachments = $hasAttachments
            importance = $importance
            inferenceClassification = $inferenceClassification
            isDraft = $isDraft
            subject = $subject
            weblink = $weblink
        }

        $sentEmails += $log
    }

    $csvObject = [PSCustomObject]@{
            FileName = $objectName
            FilePath = $objectName
            FileContent = ($sentEmails | ConvertTo-Csv) | ConvertFrom-Csv -ErrorAction SilentlyContinue
        }
    $csvObjects += $csvObject

    return $csvObjects
}


## Office Online Teams Chat

### Get Teams Chat Messages

In [44]:
function Get-OfficeOnlineTeamsChatMessages {
    param (
        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $userId
    )

    $jobs = @()
    $object = Get-StoredObject -ObjectName "MicrosoftChatMessages" -Path $storedObjectPath

    if ($object) {
        $allResults = $object
    } else {
        $msGraphUser = Get-MgUser -UserId $userId
        $userChats = Get-MgUserChat -UserId $msGraphUser.Id
    
        foreach ($chat in $userChats) {
            $jobs += Start-ThreadJob -ScriptBlock {
                param($chat, $userDisplayName)

                $chatMessages = Get-MgChatMessage -ChatId $chat.Id -ErrorAction SilentlyContinue
                $chatMessages = $chatMessages | Where-Object {$_.GetType().Name -like "MicrosoftGraphChatMessage"}

                $chatUsers = [System.Collections.Generic.HashSet[object]]::new()
                foreach ($message in $chatMessages) {
                    $chatUsers.Add($message.From.User.DisplayName) | Out-Null
                }

                $users = ($chatUsers | Where-Object {$_ -notlike $userDisplayName}) -join ","
                $messages = $chatMessages | Select-Object *, @{Name="To"; Expression={"$users"}}
                $return = $messages | Where-Object {$_.From.User.DisplayName -like $userDisplayName}

                return $return
            } -ArgumentList $chat, $userDisplayName
        }


        $allResults = @()
        foreach ($job in $jobs) {
            $chatMessages = Receive-Job -Job $job -Wait -ErrorAction SilentlyContinue
            if ($chatMessages) {
                foreach ($message in $chatMessages){
                    $allResults += $message
                }
            }
            Remove-Job -Job $job
        }
        Get-StoredObject -ObjectName "MicrosoftChatMessages" -Path $storedObjectPath -InputObject $allResults
    }

    return $allResults
}



### Convert Teams Chat Messages

In [45]:
function Convert-OfficeOnlineTeamsChatMessages {
    param (
        [Parameter(Mandatory = $true)]
        $startDateTime,

        [Parameter(Mandatory = $true)]
        $endDateTime,

        [Parameter(Mandatory = $true)]
        $csvObjects,

        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $userId
    )

    $objectName = "$userId-$($startDateTime.ToString("yyyyMMdd'T'HHmmssK"))-to-$($endDateTime.ToString("yyyyMMdd'T'HHmmssK"))-MicrosoftTeamsChats"
    $allResults = Get-OfficeOnlineTeamsChatMessages -storedObjectPath $storedObjectPath -userId $userId

    $sentChats = @()
    foreach ($row in $allResults) {
        $attachment = $row.Attachments.Content

        $doc = New-Object -TypeName HtmlAgilityPack.HtmlDocument

        if ($row.Body.Content -ne "") {
            $doc.LoadHtml($row.Body.Content)
            $docAnchors = $doc.DocumentNode.SelectNodes("//a")
            $docImgs = $imgTags = $doc.DocumentNode.SelectNodes("//img")

            $links = ""
            foreach ($anchor in $docAnchors){
                $href = $anchor.GetAttributeValue("href","")
                $text = $anchor.InnerText
                $links += "$text - $href, "
            }
            $links.TrimEnd(", ") | Out-Null

            $images = ""
            foreach ($img in $docImgs) {
                $src = $img.GetAttributeValue("src","")
                $images += "img:$src, "
            }
            $images.TrimEnd(",") | Out-Null

            $body = [System.Web.HttpUtility]::HtmlDecode(($row.Body.Content | ConvertFrom-Html).InnerText)
        } else {
            $body = ""
        }

        $log = @{
            timestamp = $row.CreatedDateTime.ToString("yyyy-MM-ddTHH:mm:ss.fffZ")
            duration = "60"
            attachment = $attachment
            links = $links
            images = $images
            body = $body
            from = $row.From.User.DisplayName
            to = $row.To.Trim(",")
            webUrl = $row.WebUrl
        }
        $sentChats += $log
    }

    $csvObject = [PSCustomObject]@{
        FileName = $objectName
        FilePath = $objectName
        FileContent = $sentChats
    }

    $csvObjects += $csvObject

    return $csvObjects
}


# Intermediate CSV Prep

### Cleaning Zero Duration Entries

In [46]:
function Remove-ZeroDurationEntries {
    param (
        [Parameter(Mandatory = $true)]
        $csvObjects
    )

    foreach ($csvObject in $csvObjects) {
        $csvObject.FileContent = $csvObject.FileContent | Where-Object {$_.duration -gt 0}
    }

    return $csvObjects
}


### Merging CSVs and Adding a 'source' column/property to each

In [47]:
function Merge-CsvContentToRows {
    param (
        [Parameter(Mandatory = $true)]
        $csvObjects
    )

    $allRows = @()
    foreach ($object in $csvObjects){
        $source = ((($object.FileName.ToString()) -replace "aw-events-export-aw-watcher-", "") -replace ".csv", "") -replace "(-\d{4}-\d{2}-\d{2})", ""
        $fileContentWithSource = $object.FileContent | Select-Object *, @{Name="Source"; Expression={"$source"}}
        $object.FileContent = $fileContentWithSource
        $allRows += $fileContentWithSource
    }

    $allHeaders = $allRows | ForEach-Object { $_.PSObject.Properties.Name } | Sort-Object -Unique

    $mergedRows = $allRows | ForEach-Object {
        $newRow = @{}
        foreach ($header in $allHeaders) {
            $newRow[$header] = $_.$header
        }
        New-Object PSObject -Property $newRow
    }
    $mergedRows = (($mergedRows | Sort-Object {[Datetime]::Parse($_.timestamp)}) | ConvertTo-Csv -NoTypeInformation) | ConvertFrom-Csv

    return $csvObjects, $mergedRows
}


### Converting to formats

#### Converting to Standard format

In [48]:
function ConvertTo-StandardCSVFormat {
    param (
        [Parameter(Mandatory = $true)]
        $rows,

        [Parameter(Mandatory = $true)]
        $email
    )

    $standardizedCsvData = $rows | ForEach-Object {
        $datetime = [DateTime]::Parse($_.timestamp)
        $formatted_datetime = $datetime.ToString("yyyy-MM-dd_HH:mm:ss")
        $startDate, $startTime = $formatted_datetime -split "_"

        $duration = [TimeSpan]::FromSeconds($_.duration)
        $formattedDuration = $duration.ToString("hh\:mm\:ss")
        
        [string]$description = "$([string]$_.Source) -"

        $headers = $_ | Get-Member -MemberType NoteProperty | Where-Object {$_.Name -notin @("Source", "timestamp", "duration")}
        
        foreach ($header in $headers){
            $description += " $(([string]$_.PSObject.Properties[$header.Name].Value).Trim())"
        }

        $standardizedRow = @{
            Duration = $formattedDuration
            "Start Date" = $startDate
            "Start Time" = $startTime
            Description = $description.Trim()
            email = $email
        }

        New-Object PSObject -Property $standardizedRow
    }

    return $standardizedCsvData
}


#### Group Consolodated Logs By Day with a max string size by character and KB

In [49]:
function Sort-RowsIntoGroupsByDay {
    param (
        [Parameter(Mandatory = $true)]
        $maxCharacterCountByBlock,

        [Parameter(Mandatory = $true)]
        $maxCharacterCountHumble,

        [Parameter(Mandatory = $true)]
        $maxBlockSizeKB,

        [Parameter(Mandatory = $true)]
        $standardizedCsvData

    )
    $maxCharacterCount = $maxCharacterCountByBlock / $maxCharacterCountHumble
    $maxSize = $maxBlockSizeKB * 1024 #KB

    $headers = $standardizedCsvData[0] | Get-Member -MemberType NoteProperty | Select-Object -ExpandProperty Name
    $csvHeadersRowString = ($headers -join ",") + "`n"
    $headerSize = [Text.Encoding]::UTF8.GetByteCount($csvHeadersRowString)
    $rows_by_date = $standardizedCsvData | Group-Object -Property "Start Date"

    $groupsByDateAndMaxCharacterCount = @{}

    foreach ($day in $rows_by_date) {
        $date = $day.Name
        $currentSubGroup = @()
        $currentSubGroupCharCount = 0
        $currentSubGroupSize = 0

        $groupsByDateAndMaxCharacterCount[$date] = @()

        foreach ($row in $day.Group) {
            $csvRow = $row | ConvertTo-Csv -NoTypeInformation -NoHeader -Delimiter ","
            $csvRowString = $csvRow -join "`n" + "`n"
            $rowSize = [Text.Encoding]::UTF8.GetByteCount($csvRowString)

            $rowCharCount = 0
            foreach ($header in $headers) {
                $rowCharCount += ($row.$header).Length
            }

            if (($currentSubGroupCharCount + $rowCharCount -le $maxCharacterCount) -and
            ($currentSubGroupSize + $rowSize -le $maxSize)) {
                    $currentSubGroup += $row
                    $currentSubGroupCharCount += $rowCharCount
                    $currentSubGroupSize += $rowSize

            } else {
                    $groupsByDateAndMaxCharacterCount[$date] += ,@($currentSubGroup)
                    $currentSubGroup = @($row)
                    $currentSubGroupCharCount = $rowCharCount
                    $currentSubGroupSize = $rowSize
            }
        }

        if ($currentSubGroup.Count -gt 0){
            $groupsByDateAndMaxCharacterCount[$date] += ,@($currentSubGroup)
        }
    }

    $groups = $groupsByDateAndMaxCharacterCount.GetEnumerator() | 
                Sort-Object { [DateTime]::ParseExact($_.Key, 'yyyy-MM-dd', $null) } |
                ForEach-Object { [PSCustomObject]@{ Name = $_.Key; Value = $_.Value } }

    return $groups
}


# LLM Processing

### Export to GPT Prompts

In [50]:
function Get-PromptHeader {
    return @"

## **Prompt**

"@
}

function Get-PromptDirective {
    return @"

Process the following log entries and generate a JSON array of ticket entries. For each ticket:

Identify the key activity or focus area.
Include relevant time entries with a start and stop time based on the logs, adjusted to be reasonable and average for the task; reduce.
Accommodate overnight tasks appropriately.
Adjust time entries to be reasonable and average for an expert at the task.

"@
}

function Get-PromptMdFenceHeader {
    return @"

``````json
"@

}

function Get-PromptJsonTemplate {
    return @"

[
    {
        "Account": "Specify the relevant client or project mentioned in the log"
        "Contacts": "List any colleagues or participants involved"
        "Status": "New"
        "Priority": "Determine based on urgency or frequency in the log"
        "IssueType": "Primary category of the activity, e.g., 'Research', 'Collaboration', 'Technical Issue'"
        "SubIssueType": "A more detailed classification if applicable"
        "EstimatedTime": "Total time spent on the activity"
        "Resources": {
            "Primary": "Yourself or the main person responsible"
            "Secondary": "Additional colleagues involved"
        }
        "ToolsAndServicesUsed" = {
            "Tools": "List any tools, software, or departments involved"
            "URLs": ["A list of collected URLs found for the ticket"]
        }
        "TimeEntries" = [
            {
                "start_time": "yyyy-MM-ddTHH:mm:ss.fffZ"
                "end_time": "yyyy-MM-ddTHH:mm:ss.fffZ"
                "description": "A detailed summary including Key tasks performed or searches conducted and major insights gained"
            },
            {
                "start_time": "yyyy-MM-ddTHH:mm:ss.fffZ"
                "end_time": "yyyy-MM-ddTHH:mm:ss.fffZ"
                "description": "A detailed summary including Key tasks performed or searches conducted and major insights gained"
            },
            {
                ...
            }
        ]
        "Title": "A brief, unique summary of the activity"
        "Project": "Name of the Project"
        "Description": "A detailed summary including Key tasks performed or searches conducted and major insights gained"
        "Billable": "Assess whether this should be classified as billiable or unbillable."
    },
    {
        ...
    }
]

"@
}

function Get-PromptMdFenceFooter {
    return @"

``````

"@
}

function Get-PromptJsonDirectiveTrailer {
    return @"

Don't use any markdown backticks, additional text, explanations, or formatting. Output only the JSON data.
Don't use any markdown backticks, additional text, explanations, or formatting. Output only the JSON data.

"@
}

function Get-PromptDataHeader {
    return @"

### Data

"@
}

function Generate-Prompts {
    param (
        [Parameter(Mandatory = $true)]
        $fileBaseName,

        [Parameter(Mandatory = $true)]
        $groups
    )

    $prompts = @()
    foreach ($date in $groups){

        $dateCsvCount = $date.Value.Count
        $csvCount = 0
        foreach ($csv in $date.Value){
            $csvCount++ 
            $prompt_data = $null


            $fileName = "$fileBaseName-for-$($date.Name)-part-$csvCount-of-$dateCsvCount.csv"

            $csv | Export-Csv -Path ".\StoredObjects\CSVOut\$filename" -NoTypeInformation

            $csvString = ($csv | ConvertTo-Csv -NoTypeInformation) -join "`n"

            $promptCritia = @((Get-PromptHeader), (Get-PromptDirective), (Get-PromptMdFenceHeader), (Get-PromptJsonTemplate), (Get-PromptMdFenceFooter), (Get-PromptJsonDirectiveTrailer), (Get-PromptDataHeader)) -join ""

            $prompt = $promptCritia + $csvString
            $prompts += $prompt
        }
    }

    return $prompts
}


## Getting JSON Tickets from OpenAI

### Get Model

In [51]:
function Get-Gpt4oLatestModel {
    $openAiApiCreds = Get-StoredCredential -CredentialName "openAiApiCreds" -Path ".\StoredCredentials"
    $listModelsEndpoint = "https://api.openai.com/v1/models"
    $headers = @{
        "Authorization" = "Bearer $($openAiApiCreds.GetNetworkCredential().Password)"
        "Content-Type" = "application/json"
    }
    $chatgpt4oId = ((Invoke-RestMethod -Uri $listModelsEndpoint -Method Get -Headers $headers).data | Where-Object {$_.id -like "chatgpt-4o-latest"}).id

    if ($chatgpt4oId.Length -gt 0) {
        return $chatgpt4oId
    } else {
        throw [System.ArgumentException]::new("Parameter cannot be null.", "`$chatgpt4oId")
    }
}


### Call API and Get Completions

In [52]:
function Get-Gpt4oCompletions {
    param (
        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $prompts,

        $responseCount = 0
    )

    
    $completionsEndpoint = "https://api.openai.com/v1/chat/completions"

    $object = Get-StoredObject -ObjectName "OpenAiCompletions" -Path $storedObjectPath
    if ($object) {
        $allResults = $object
    } else {
        $openAiApiCreds = Get-StoredCredential -CredentialName "openAiApiCreds" -Path ".\StoredCredentials"
        

        $jobs = @()
        $count = 0
        foreach ($prompt in $prompts) {
            $count++
            $jobs += Start-ThreadJob -ScriptBlock{
                param($prompt, $completionsEndpoint, $openAiApiCreds, $chatgpt4oId, $headers, $storedObjectPath)
                . ".\Powershell\Get-StoredObject.ps1" | Out-Null

                $headers = @{
                    "Authorization" = "Bearer $($openAiApiCreds.GetNetworkCredential().Password)"
                    "Content-Type" = "application/json"
                }
                $promptHash = [System.BitConverter]::ToString([System.Security.Cryptography.SHA256]::Create().ComputeHash([System.Text.Encoding]::UTF8.GetBytes($prompt))).Replace("-", "")
                $completion = Get-StoredObject -ObjectName $promptHash -Path "$storedObjectPath\Completions"

                if ($completion) {
                    return $completion
                } else {
                    $requestBody = @{
                        model = $chatgpt4oId
                        messages = @(
                            @{
                                role = "user"
                                content = $prompt
                            }
                        )
                        temperature = 0.7
                        max_tokens = 1024
                        top_p = 0.9
                        frequency_penalty = 0.0
                    } | ConvertTo-Json -Depth 6 -Compress

                    Start-Sleep -Seconds 15
                    $completion = Invoke-RestMethod -Uri $completionsEndpoint -Method Post -Headers $headers -Body $requestBody
                    
                    $return = [pscustomobject]@{
                        content = $completion.choices[0].message.content
                        promptHash = $promptHash
                        prompt = $prompt
                    }
                    Get-StoredObject -ObjectName $promptHash -Path "$storedObjectPath\Completions" -InputObject $return
                    return $return
                }

            } -ArgumentList $prompt, $completionsEndpoint, $openAiApiCreds, (Get-Gpt4oLatestModel), $headers, $storedObjectPath
            
            Start-Sleep -Seconds 15
            if($responseCount -gt 0){
                if ($count -gt ($responseCount - 1)){break}
            }
        }

        $allResults = @()
        foreach ($job in $jobs) {
            $completions = Receive-Job -Job $job -Wait
            if ($completions) {
                foreach ($result in $completions){
                    $allResults += $result
                    
                }
            }
            Remove-Job -Job $job
        }
        
        Get-StoredObject -ObjectName "OpenAiCompletions" -Path $storedObjectPath -InputObject $allResults
        
    }

    return $allResults
}


### Cleaning Responses

#### Grouping Valid Json

In [53]:
function Group-ValidJsonCompletions {
    param (
        [Parameter(Mandatory = $true)]
        $jsonCompletions
    )

    $validJson = @()
    $invalidJson = @()

    foreach ($result in $jsonCompletions) {
        try {
            # Attempt to parse JSON content using .NET's JSON parser
            [System.Text.Json.JsonDocument]::Parse($result.content) | Out-Null

            # Add to valid JSON collection if parsing is successful
            $completion = [PSCustomObject]@{
                promptHash = $result.promptHash
                content = $result.content
                prompt = $result.prompt
            }
            $validJson += $completion
        } catch {
            # Add to invalid JSON collection if parsing fails, capturing error message
            $completion = [PSCustomObject]@{
                promptHash = $result.promptHash
                content = $result.content
                prompt = $result.prompt
                jsonError = $_.Exception.Message
            }
            $invalidJson += $completion
        }
    }
    return $validJson, $invalidJson
}


#### Repairing Json

##### Generating all Json Tail Closures

In [54]:
function Generate-AllJsonClosurePermutations {
    param (
        [int]$MaxDepth  # The maximum depth for permutations
    )

    # Initialize an array to hold all permutations
    $allPermutations = @()  

    function Generate-Permutations {
        param (
            [int]$Depth,           # The target depth of the current call
            [string]$Current = "",  # The current permutation string being built
            [ref]$Permutations     # A reference to the main array to store permutations
        )

        # Base case: if the current depth is reached, add the permutation to the array
        if ($Current.Length -eq $Depth) {
            # Add the permutation to the array passed by reference
            $Permutations.Value += $Current  
            return
        }

        # Recursive case: append each character `]` and `}` and go deeper
        Generate-Permutations -Depth $Depth -Current ($Current + "]") -Permutations $Permutations
        Generate-Permutations -Depth $Depth -Current ($Current + "}") -Permutations $Permutations
    }

    # Generate permutations for each depth from 1 to MaxDepth
    for ($currentDepth = 1; $currentDepth -le $MaxDepth; $currentDepth++) {
        Generate-Permutations -Depth $currentDepth -Permutations ([ref]$allPermutations)
    }

    # Return the array of permutations
    return $allPermutations
}

function Generate-JsonTailRepairs {
    $permutations = Generate-AllJsonClosurePermutations -MaxDepth 6

    $repair_prefixes = ('":""', ':""', '""', '"')

    foreach ($permutation in $permutations) {
        foreach ($prefix in $repair_prefixes) {
            $permutations += $prefix + $permutation
        }
    }

    $tailRepairs = $permutations | Sort-Object {$_.Length}

    return $tailRepairs
}


##### Non-Destructive Json Tail Closure Fix

In [55]:
function RepairWith-NonDestructiveTail ($promptHash, $content, $prompt, $jsonError) {
    $tailRepairs = (Generate-JsonTailRepairs)

    foreach ($repair in $tailRepairs) {
        try {
            $repaired = (($content + $repair) | ConvertFrom-Json) | ConvertTo-Json -Depth 6

            if ($repaired) {
                $repairedJson = [PSCustomObject]@{
                    promptHash = $promptHash
                    content = $content
                    prompt = $prompt
                    jsonError = $jsonError
                    repairedByNonDestructiveTail = $true
                    repairedJson = $repaired
                }

                return $repairedJson
                break
            }
        } catch {}
    }

    return $false
}


##### Last Comma - Destructive Json Tail Closure Fix

In [56]:
function RepairWith-LastCommaTail ($promptHash, $content, $prompt, $jsonError) {
    $tailRepairs = (Generate-JsonTailRepairs)
    
    $parts = $content -split ","
    $before = ($parts[0..($parts.Length - 2)] -join ",")
    $after = ("," + $parts[-1])
    foreach ($repair in $tailRepairs) {
        try {
            $repaired = (($before + $repair) | ConvertFrom-Json) | ConvertTo-Json -Depth 6

            if ($repaired) {
                $repairedJson = [PSCustomObject]@{
                    promptHash = $promptHash
                    content = $content
                    prompt = $prompt
                    jsonError = $jsonError
                    repairedByLastCommaTail = $true
                    repairedJson = $repaired
                    parts = $parts
                    before = $before
                    after = $after
                }

                return $repairedJson
                break
            }
        } catch {}
    }

    return $false
}


##### Remove GPT Markdown Fense Artifacts

In [57]:
function RepairWith-RemoveMarkdownFense ($promptHash, $content, $prompt, $jsonError) {
    if (($content[0..10] -join "")-like "*``````json*") {
        $trimStart = $content.TrimStart("``````json")
        $repairedMarkdown = $trimStart.TrimEnd("``````")

        try {
            $repaired = ($repairedMarkdown | ConvertFrom-Json) | ConvertTo-Json -Depth 6

            if ($repaired) {
                $repairedJson = [pscustomobject]@{
                    promptHash = $promptHash
                    content = $content
                    prompt = $prompt
                    jsonError = $jsonError
                    appliedRemoveMarkdownFense = $true
                    repairedByRemoveMarkdownFense = $true
                    repairedMarkdown = $repairedMarkdown
                    repairedJson = $repaired
                }

                return $repairedJson
            }
        } catch {
            $appliedMarkdownFense = [PSCustomObject]@{
                promptHash = $promptHash
                content = $content
                prompt = $prompt
                jsonError = $jsonError
                appliedRemoveMarkdownFense = $true
                repairedByRemoveMarkdownFense = $false
                repairedMarkdown = $repairedMarkdown
            }
            
            return $appliedMarkdownFense    
        }
    }

    return $false
}


##### If a Value should be an array, repair it

In [58]:

function RepairWith-JsonLineArray ($promptHash, $content, $prompt, $jsonError) {

    $jsonError -match 'LineNumber:\s*(\d+)\s*\|' | Out-Null
    $lineNumber = $Matches[1]
    $lines = $content -split "`r?`n"
    $string = $lines[$lineNumber - 1]

    $string

    try {
        if ($lines[$lineNumber - 1] -match ":") {
            $keyValue = $lines[$lineNumber - 1] -split ":"
            $arrayRepairedJson = @($keyValue[0], (("[$($keyValue[1])]" | ConvertFrom-Json) | ConvertTo-Json -Compress)) -join ":"
            
            $lines[$lineNumber -1] = $arrayRepairedJson
            $repairedArray = $lines -join "`n"

            try {
                $repaired = ($repairedArray | ConvertFrom-Json) | ConvertTo-Json -Depth 6

                if ($repaired) {
                    $repairedJson = [pscustomobject]@{
                        promptHash = $promptHash
                        content = $content
                        prompt = $prompt
                        jsonError = $jsonError
                        appliedJsonLineArrayFix = $true
                        repairedByJsonLineArrayFix = $true
                        repairedArray = $repairedArray
                        repairedJson = $repaired
                    }

                    return $repairedJson
                }
            } catch {
                $repairedJsonLineArray = [PSCustomObject]@{
                    promptHash = $promptHash
                    content = $content
                    prompt = $prompt
                    jsonError = $jsonError
                    appliedJsonLineArrayFix = $true
                    repairedByJsonLineArrayFix = $false
                    repairedArray = $repairedArray
                }
                
                return $repairedJsonLineArray
            }
        } else {
            throw
        }
    } catch {
        return $false
    }
}


#### Apply All Json Repairs

In [59]:
function Repair-JsonCompletions {
    param (
        [Parameter(Mandatory = $true)]
        $invalidJson
    )
    $RepairedJson = @()
    $RepairFailures = @()
    foreach ($json in $invalidJson) {
        $promptHash = $json.promptHash
        $content = $json.content
        $prompt = $json.prompt
        $jsonError = $json.jsonError

        $nonDestructiveTailRepair = RepairWith-NonDestructiveTail -promptHash $promptHash -content $content -prompt $prompt -jsonError $jsonError
        

        if ($nonDestructiveTailRepair) {
            $RepairedJson += $nonDestructiveTailRepair
        } else {
            switch -Wildcard ($jsonError) {
                '*"Expected end of string, but instead reached end of data.*' {

                    $lastCommaTailRepair = RepairWith-LastCommaTail -promptHash $promptHash -content $content -prompt $prompt -jsonError $jsonError

                    if ($lastCommaTailRepair) {
                        $RepairedJson += $lastCommaTailRepair
                    } else {
                        $notRepairedJson = [pscustomobject]@{
                            promptHash = $promptHash
                            content = $content
                            prompt = $prompt
                            jsonError = $jsonError
                        }

                        $RepairFailures += $notRepairedJson
                    }
                }

                "*`' is an invalid start of a value.*" {
                    $repairedMarkdownFense = RepairWith-RemoveMarkdownFense -promptHash $promptHash -content $content -prompt $prompt -jsonError $jsonError
                    
                    if ($repairedMarkdownFense) {
                        if($repairedMarkdownFense.repairedByRemoveMarkdownFense -eq $true) {
                            $RepairedJson += $repairedMarkdownFense
                        } else {
                            
                            $nonDestructiveTailRepair = RepairWith-NonDestructiveTail -promptHash $promptHash -content $repairedMarkdownFense.repairedMarkdown -prompt $prompt -jsonError $jsonError

                            if ($nonDestructiveTailRepair) {

                                $RepairedJson += [PSCustomObject]@{
                                    promptHash = $promptHash
                                    content = $content
                                    prompt = $prompt
                                    jsonError = $jsonError
                                    repairedByNonDestructiveTail = $nonDestructiveTailRepair.repairedByNonDestructiveTail
                                    repairedJson = $nonDestructiveTailRepair.repairedJson
                                    appliedRemoveMarkdownFense = $repairedMarkdownFense.appliedRemoveMarkdownFense
                                    repairedByRemoveMarkdownFense = $repairedMarkdownFense.repairedByRemoveMarkdownFense
                                    repairedMarkdown = $repairedMarkdownFense.repairedMarkdown
                                }
                            } else {
                                $lastCommaTailRepair = RepairWith-LastCommaTail -promptHash $promptHash -content $repairedMarkdownFense.repairedMarkdown -prompt $prompt -jsonError $jsonError

                                if ($lastCommaTailRepair) {
                                    $RepairedJson += [PSCustomObject]@{
                                        promptHash = $promptHash
                                        content = $content
                                        prompt = $prompt
                                        jsonError = $jsonError
                                        repairedByLastCommaTail = $lastCommaTailRepair.repairedByLastCommaTail
                                        repairedJson = $lastCommaTailRepair.repairedJson
                                        parts = $lastCommaTailRepair.parts
                                        before = $lastCommaTailRepair.before
                                        after = $lastCommaTailRepair.after
                                        appliedRemoveMarkdownFense = $repairedMarkdownFense.appliedRemoveMarkdownFense
                                        repairedByRemoveMarkdownFense = $repairedMarkdownFense.repairedByRemoveMarkdownFense
                                        repairedMarkdown = $repairedMarkdownFense.repairedMarkdown
                                    }
                                } else {
                                    $notRepairedJson = [pscustomobject]@{
                                        promptHash = $promptHash
                                        content = $content
                                        prompt = $prompt
                                        jsonError = $jsonError
                                    }

                                    $RepairFailures += $notRepairedJson
                                }
                            }
                        }
                    } else {
                        $notRepairedJson = [pscustomobject]@{
                            promptHash = $promptHash
                            content = $content
                            prompt = $prompt
                            jsonError = $jsonError
                        }

                        $RepairFailures += $notRepairedJson
                    }
                
                }

                "*}' is invalid after a property name. Expected a ':*" {
                    $repairedJsonLineArray = RepairWith-JsonLineArray -promptHash $promptHash -content $content -prompt $prompt -jsonError $jsonError

                    if ($repairedJsonLineArray) {
                        if($repairedJsonLineArray.repairedByJsonLineArrayFix -eq $true) {
                            $RepairedJson += $repairedJsonLineArray
                        } else {
                            $nonDestructiveTailRepair = RepairWith-NonDestructiveTail -promptHash $promptHash -content $repairedJsonLineArray.repairedArray -prompt $prompt -jsonError $jsonError

                            if ($nonDestructiveTailRepair) {

                                $RepairedJson += [PSCustomObject]@{
                                    promptHash = $promptHash
                                    content = $content
                                    prompt = $prompt
                                    jsonError = $jsonError
                                    repairedByNonDestructiveTail = $nonDestructiveTailRepair.repairedByNonDestructiveTail
                                    repairedJson = $nonDestructiveTailRepair.repairedJson
                                    appliedJsonLineArrayFix = $repairedJsonLineArray.appliedRemoveMarkdownFense
                                    repairedByJsonLineArrayFix = $repairedJsonLineArray.repairedByRemoveMarkdownFense
                                    repairedArray = $repairedJsonLineArray.repairedMarkdown
                                }
                            } else {
                                $lastCommaTailRepair = RepairWith-LastCommaTail -promptHash $promptHash -content $repairedJsonLineArray.repairedArray -prompt $prompt -jsonError $jsonError

                                if ($lastCommaTailRepair) {
                                    $RepairedJson += [PSCustomObject]@{
                                        promptHash = $promptHash
                                        content = $content
                                        prompt = $prompt
                                        jsonError = $jsonError
                                        repairedByLastCommaTail = $lastCommaTailRepair.repairedByLastCommaTail
                                        repairedJson = $lastCommaTailRepair.repairedJson
                                        parts = $lastCommaTailRepair.parts
                                        before = $lastCommaTailRepair.before
                                        after = $lastCommaTailRepair.after
                                        appliedJsonLineArrayFix = $repairedJsonLineArray.appliedRemoveMarkdownFense
                                        repairedByJsonLineArrayFix = $repairedJsonLineArray.repairedByRemoveMarkdownFense
                                        repairedArray = $repairedJsonLineArray.repairedMarkdown
                                    }
                                } else {
                                    $notRepairedJson = [pscustomobject]@{
                                        promptHash = $promptHash
                                        content = $content
                                        prompt = $prompt
                                        jsonError = $jsonError
                                    }

                                    $RepairFailures += $notRepairedJson
                                }
                            }
                        }
                    } else {
                        $notRepairedJson = [pscustomobject]@{
                            promptHash = $promptHash
                            content = $content
                            prompt = $prompt
                            jsonError = $jsonError
                        }

                        $RepairFailures += $notRepairedJson
                    }
                }

                default {
                    $notRepairedJson = [pscustomobject]@{
                        promptHash = $promptHash
                        content = $content
                        prompt = $prompt
                        jsonError = $jsonError
                    }

                    $RepairFailures += $notRepairedJson
                }
            }
        }
    }
    if ($invalidJson.Count -eq $RepairedJson.Count) {
        Write-Host "Repaired $($RepairedJson.Count)/$($invalidJson.Count) of invalid json objects."
        Write-Host "Repaired $($RepairedJson.Count)/$($validJson.Count + $invalidJson.Count) of total json objects."
    }

    return $RepairedJson, $RepairFailures
}


#### Review Results

In [60]:

function Generate-ContinuePrompts {
    param(
        [Parameter(Mandatory = $true)]
        $repairedJson
    )

    $trunkatedCompletions = $repairedJson | Where-Object { 
        $_.PSObject.Properties['repairedByNonDestructiveTail'] -ne $null -or 
        $_.PSObject.Properties['repairedByLastCommaTail'] -ne $null 
    }


    $prompts = @()
    $propertyName = "continuePrompt"
    $mdFenceRepairedCompletions = $trunkatedCompletions | Where-Object {
        $_.PSObject.Properties['appliedRemoveMarkdownFense'] -ne $null
    }
    foreach ($completion in $mdFenceRepairedCompletions) {
        $continueCompletion = @{}
        foreach ($property in $completion.PSObject.Properties) {
            $continueCompletion[$property.Name] = $property.Value
        }
        $continuePrompt = (@($completion.prompt, "### Reply", $completion.repairedMarkdown, "`ncontinue`n") -join "`n`n")
        $continueCompletion[$propertyName] = $continuePrompt

        $prompts += [PSCustomObject]$continueCompletion
    }

    $arrayRepairedCompletions = $trunkatedCompletions | Where-Object {
        $_.PSObject.Properties['appliedJsonLineArrayFix'] -ne $null
    }
    foreach ($completion in $arrayRepairedCompletions) {
        $continueCompletion = @{}
        foreach ($property in $completion.PSObject.Properties) {
            $continueCompletion[$property.Name] = $property.Value
        }
        $continuePrompt = (@($completion.prompt, "### Reply", $completion.repairedArray, "`ncontinue`n") -join "`n`n")
        $continueCompletion[$propertyName] = $continuePrompt

        $prompts += [PSCustomObject]$continueCompletion
    }

    $lastCommaRepairedCompletions = $trunkatedCompletions | Where-Object {
        ($_.PSObject.Properties['repairedByNonDestructiveTail'] -eq $null) -and
        ($mdFenceRepairedCompletions -notcontains $_) -and
        ($arrayRepairedCompletions -notcontains $_)
    }
    foreach ($completion in $lastCommaRepairedCompletions) {
        $continueCompletion = @{}
        foreach ($property in $completion.PSObject.Properties) {
            $continueCompletion[$property.Name] = $property.Value
        }
        $continuePrompt = (@($completion.prompt, "### Reply", $completion.before, "`ncontinue`n") -join "`n`n")
        $continueCompletion[$propertyName] = $continuePrompt

        $prompts += [PSCustomObject]$continueCompletion
    }

    $nonDestructiveRepairedCompletions = $trunkatedCompletions | Where-Object {
        ($_.PSObject.Properties['repairedByLastCommaTail'] -eq $null) -and
        ($mdFenceRepairedCompletions -notcontains $_) -and
        ($arrayRepairedCompletions -notcontains $_)
    }
    foreach ($completion in $nonDestructiveRepairedCompletions){
        $continueCompletion = @{}
        foreach ($property in $completion.PSObject.Properties) {
            $continueCompletion[$property.Name] = $property.Value
        }
        $continuePrompt = (@($completion.prompt, "### Reply", $completion.content, "`n(continue)`n") -join "`n`n")
        $continueCompletion[$propertyName] = $continuePrompt

        $prompts += [PSCustomObject]$continueCompletion
    }
    return $prompts
}


In [61]:
function Get-Gpt4oContinueCompletions {
    param (
        [Parameter(Mandatory = $true)]
        $storedObjectPath,

        [Parameter(Mandatory = $true)]
        $prompts,

        $responseCount = 0
    )

    
    $completionsEndpoint = "https://api.openai.com/v1/chat/completions"

    $object = Get-StoredObject -ObjectName "OpenAiContinueCompletions" -Path $storedObjectPath
    if ($object) {
        $allResults = $object
    } else {
        $openAiApiCreds = Get-StoredCredential -CredentialName "openAiApiCreds" -Path ".\StoredCredentials"
        
        $jobs = @()
        $count = 0
        foreach ($prompt in $prompts) {
            $count++
            $jobs += Start-ThreadJob -ScriptBlock{
                param($prompt, $completionsEndpoint, $openAiApiCreds, $chatgpt4oId, $headers, $storedObjectPath)
                . ".\Powershell\Get-StoredObject.ps1" | Out-Null

                $headers = @{
                    "Authorization" = "Bearer $($openAiApiCreds.GetNetworkCredential().Password)"
                    "Content-Type" = "application/json"
                }
                $promptHash = [System.BitConverter]::ToString([System.Security.Cryptography.SHA256]::Create().ComputeHash([System.Text.Encoding]::UTF8.GetBytes($prompt.continuePrompt))).Replace("-", "")
                $completion = Get-StoredObject -ObjectName $promptHash -Path "$storedObjectPath\Completions\Continue"

                if ($completion) {
                    return $completion
                } else {
                    $requestBody = @{
                        model = $chatgpt4oId
                        messages = @(
                            @{
                                role = "user"
                                content = $prompt.continuePrompt
                            }
                        )
                        temperature = 0.7
                        max_tokens = 1024
                        top_p = 0.9
                        frequency_penalty = 0.0
                    } | ConvertTo-Json -Depth 6 -Compress

                    Start-Sleep -Seconds 15
                    $completion = Invoke-RestMethod -Uri $completionsEndpoint -Method Post -Headers $headers -Body $requestBody
                    
                    $continueCompletion = @{}
                    foreach ($property in $prompt.PSObject.Properties) {
                        $continueCompletion[$property.Name] = $property.Value
                    }
                    $continueCompletion["continueContent"] = @(,$completion.choices[0].message.content)

                    $return = [PSCustomObject]$continueCompletion
                    Get-StoredObject -ObjectName $promptHash -Path "$storedObjectPath\Completions\Continue" -InputObject $return
                    return $return
                }

            } -ArgumentList $prompt, $completionsEndpoint, $openAiApiCreds, (Get-Gpt4oLatestModel), $headers, $storedObjectPath
            
            Start-Sleep -Seconds 15
            if($responseCount -gt 0){
                if ($count -gt ($responseCount - 1)){break}
            }
        }

        $allResults = @()
        foreach ($job in $jobs) {
            $completions = Receive-Job -Job $job -Wait
            if ($completions) {
                foreach ($result in $completions){
                    $allResults += $result
                    
                }
            }
            Remove-Job -Job $job
        }
        
        Get-StoredObject -ObjectName "OpenAiContinueCompletions" -Path $storedObjectPath -InputObject $allResults
        
    }

    return $allResults
}


In [ ]:
$csvObjects = Import-ActivityWatchCSVs -activitywatchDirectory ".\activitywatch_exports"
$csvObjects = Repair-ChildActivityWatchObsidianCSV -csvObjects $csvObjects
$csvObjects = Filter-ActivityWatchByAfkStatus -activityWatchCSVsObject $csvObjects -storeObjectPath ".\StoredObjects\ActivityWatch" -afkCSVsDirectory ".\activitywatch_exports_afk"
$sortedDateRanges = Get-SortedDateRanges -startDateTime ([datetime]::ParseExact("24-09-04", "yy-MM-dd", $null)) -endDateTime ([datetime]::Today)
$csvObjects = Add-DattoRemoteActivityCSVs -dattoUserId "99708" -storedObjectPath ".\StoredObjects\datto" -csvObjects $csvObjects -dateRangesObject $sortedDateRanges
$csvObjects = Convert-OfficeOnlineCalendarEvents -startDateTime ([datetime]::ParseExact("24-09-04", "yy-MM-dd", $null)) -endDateTime ([datetime]::Today) -csvObjects $csvObjects -storedObjectPath ".\StoredObjects\Microsoft" -dateRangesObject $sortedDateRanges -userId "david@avartec.com"
$csvObjects = Convert-OfficeOnlineSentEmails -startDateTime ([datetime]::ParseExact("24-09-04", "yy-MM-dd", $null)) -endDateTime ([datetime]::Today) -csvObjects $csvObjects -storedObjectPath ".\StoredObjects\Microsoft" -dateRangesObject $sortedDateRanges -userId "david@avartec.com"
$csvObjects = Convert-OfficeOnlineTeamsChatMessages -startDateTime ([datetime]::ParseExact("24-09-04", "yy-MM-dd", $null)) -endDateTime ([datetime]::Today) -csvObjects $csvObjects -storedObjectPath ".\StoredObjects\Microsoft" -userId "david@avartec.com"
$csvObjects = Remove-ZeroDurationEntries -csvObjects $csvObjects
$csvObjects, $mergedRows = Merge-CsvContentToRows -csvObjects $csvObjects
$standardizedCsvData = ConvertTo-StandardCSVFormat -rows $mergedRows -email "david@avartec.com"
$dataGroups = Sort-RowsIntoGroupsByDay -maxCharacterCountByBlock 55000 -maxCharacterCountHumble 1.5 -maxBlockSizeKB 10000 -standardizedCsvData $standardizedCsvData
$prompts = Generate-Prompts -fileBaseName "time_entries" -groups $dataGroups


# $completions = Get-Gpt4oCompletions -storedObjectPath ".\StoredObjects\OpenAi" -prompts $prompts # -responseCount 1
# $validJsonCompletions, $invalidJsonCompletions = Group-ValidJsonCompletions -jsonCompletions $completions
# $repairedJson, $repairFailures = Repair-JsonCompletions -invalidJson $invalidJsonCompletions
# $continuePrompts = Generate-ContinuePrompts -repairedJson $repairedJson
# $continueCompletions = Get-Gpt4oContinueCompletions -storedObjectPath ".\StoredObjects\OpenAi" -prompts $continuePrompts # -responseCount 1


In [ ]:
Write-Host "Collected Data Objects: $($csvObjects.Count)"
Write-Host "Rows Merged from collected data objects: $($mergedRows.Count)"
Write-Host "Rows of Standardized Columnar Data: $($standardizedCsvData.Count)"
Write-Host "Days of Collection: $($dataGroups.Count)"
Write-host "Number of Prompts to Send: $($prompts.Count)"
# Write-Host "Number of Completions Recieved: $($completions.Count)"
# Write-Host "Count of Valid Json completion responses on reciept: $($validJsonCompletions.Count)"
# Write-Host "Count of Invalid Json completion responses on reciept: $($invalidJsonCompletions.Count)"
# Write-Host "Count of Invalid Json that was able to be repaired: $($repairedJson.Count)"
# Write-Host "Count of Invalid Json that was NOT able to be repaired: $($repairFailures.Count)"
# Write-host "Number of Continue Prompts for truncated completions to be requested: $($continuePrompts.Count)"
# Write-Host "Number of Continue Completions Recieved: $($continueCompletions.Count)"


In [ ]:
$prompts[0]


In [ ]:
# $continueCompletions[100].before
foreach ($completion in $continueCompletions) {
    $first = ($completion.continueContent[0][0..6]) -join ""

    $switch = $first -replace '\s+', ''
    
    $break = $false
    switch ($switch) {
        "[{" {

            $completion.before
            write-host "---------------------------------------------"
            $completion.continueContent[0]

            $sample = $completion
            $break = $true
            break
        }

        "``````json" {

        }

        "{" {}

        "[" {

        }

        "" {

        }

        default {
        }
    }
    if($break) {break}
}


In [ ]:
$sample | Get-Member -MemberType NoteProperty | Select-Object -ExpandProperty Name
